# Bug Fixes:
- Saved Year Make and Model in the First API Call instead of soley in the second API call, in case the first API call works but the second one fails
- Removed any potential whitespaces from license plate numbers, at the beginning of the loop before they enter the API call
- Removed any potential whitespaces from the returned VIN value from JSON response, to prevent HTTP 400 Bad Request Errors (not 100% sure this is the actual cause)
- The ability to loop through all 5 states for a license plate before moving on to the next license plate 
- Added statements for errors, Success getting 1st / 2nd API call, and when it moves onto the next plate.
- Added empty dictionaries for each column so that when we try to pull data at the end of the loop into Pandas Dataframe, it doesn't cause any errors with the hardcoded column formatting.
- Added retry attempts for HTTP 400 Bad Requests on VIN -> Market Value in Step 3.
- License plates are automatically saved into snowflake after each iteration.
- Added in logic in SQL code to only pull in revenues that are positive to avoid grabbing refund / cancelation transactions
- Added in logic to ensure that the license plates inputed into the API loop from SQL only selects the previous month and not all transactions
- Added in a progress bar
- Set up secret in Snowflake with actual API key
- Fine tune timeout amount to avoid HTTP 429 Too Many Requests


# (Part 1) Establish a Snowflake connection and pulling in data from PROD_GOLD.PARKING.DIM_PARKINGCUSTOMER

In [ ]:
from tqdm import tqdm
import streamlit as st 
import requests
import snowflake.connector
import snowflake.snowpark.functions as F
import pandas as pd
import os
import json
import time
import traceback
import re
from datetime import date
from pathlib import Path 
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.dataframe_writer import SaveMode
TABLE_NAME = "VEHICLE_DATABASES_API_RESULTS_DATA"
today = date.today()

session = get_active_session()

# Set the Snowflake Database we want to pull data from so we don't have to repeatedly type it out
session.use_database('DEV_BRONZE')

# Set the specific Schmea we want to use
session.use_schema('PARKING')

# Joins the first 400 rows of data from Chauntry, SKIDATA, and OCR and filters out any plates that has already been ran before

license_plate_ingest_sql = """
WITH already_processed as(
select PLATE as LICENSE_PLATE_NUMBER
from DEV_BRONZE.PARKING.VEHICLE_DATABASES_API_RESULTS_DATA
),

LICENSE_FILTERED as (
SELECT r.*
FROM DEV_BRONZE.PARKING.MONTHLY_TXN_SAMPLE as r
LEFT JOIN already_processed as p
on r.plateno = p.license_plate_number
where p.license_plate_number is null
)

select plateno as license_plate_number
from license_filtered
where DATE_TRUNC('month', CURRENT_DATE) = DATE_TRUNC('month', IMPORT_DATE);
"""

plate_df = session.sql(license_plate_ingest_sql).to_pandas() # Turns our SQL into a Pandas dataframe

# Printing a sample of the license plate data with number of rows / plates to be ran


In [40]:
# Check to ensure the data from Snowflake contains correct number of rows and data
print(plate_df.head(2000))


    LICENSE_PLATE_NUMBER
0                9CAH166
1                6VMS200
2                7ZDD095
3                6HDA953
4                5ZGG746
..                   ...
847              819CNE3
848              8UZN206
849              9CSJ747
850              ATORITO
851              9STW963

[852 rows x 1 columns]


# (Part 2) Loads in API Key Function
loads in the API Key and saves it as a dictionary

In [ ]:
def load_api_key_dict() -> Dict[str, str]:
    """
    Reads VEHICLE_API_KEY from secrets
    Plain string: "..."                -> wrapped into {"x-AuthKey": "..."}
    """
    import streamlit as st
    raw = st.secrets["VEHICLE_API_KEY"]  
    return {"x-AuthKey": raw}

api_key_dict = load_api_key_dict()
AUTH_KEY = api_key_dict["x-AuthKey"]

# (Part 3) Save / Upload Function
Creating a function that will automatically save a license plate to Snowflake table "DEV_BRONZE.PARKING.VEHICLE_DATABASES_API_RESULTS_DATA" after each iteration. This ensures that each plate that is ran is automatically saved and if for some reason the API notebook failed the data would already be in the results table and if the notebook was restarted the plates already ran would be filtered out, ensuring no duplicate plates would be ran and no credits would be wasted.

In [ ]:
def save_plate(row_dict: dict) -> None:
    """
    Append a single record to DEV_BRONZE.PARKING.VEHICLE_DATABASES_API_RESULTS_DATA.
    Creates the table on first call.
    """
    df = pd.DataFrame([row_dict])                # one-row DataFrame

    # cast YEAR to nullable integer if the key exists
    if "YEAR" in df.columns:
        df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce").astype("Int64")

    session.write_pandas(
        df,
        table_name = TABLE_NAME,
        auto_create_table = True,                # first run
        save_mode = SaveMode.APPEND              # every run
    )

# (Part 3) Vehicle Market Value Function
Creating a function that will do part-two of the License Plate -> Market Value process. The function will take a VIN and input it into Vehicle Databases API and return a list of market values. The code will then loop through the nested dictionaries and return the "Average Condition Dealer Retail" value 

In [ ]:
import requests, re
from typing import Optional, Dict

# This function removes $ dollar signs and , commas from our returned vehicle market value, and ensures the number is an integer $43,221 -> 43221
def dollars_to_int(s: str) -> int:
    return int(re.sub(r"[^\d]", "", s))

# This function is our second API call that takes the returned VIN from License Plate and uses the API to return a Vehicle Market Value along with Vehicle Data
def get_market_value(vin: str, auth_key: Optional[str] = None) -> Dict[str, object]:
    # Fill auth_key from our normalized dict if not provided
    if auth_key is None:
        auth_key = api_key_dict["x-AuthKey"]

    vin_clean = re.sub(r"\s+", "", vin)
    
    # ________________ HTTP REQUEST _____________________________
    BASE_URL = "https://api.vehicledatabases.com/market-value/v2"
    url      = f"{BASE_URL}/{vin_clean}?state={None}&mileage={None}"
    headers = {"x-AuthKey": auth_key}
    
    r = requests.get(url, headers=headers, timeout=20)

    info = r.json() # Takes the returned JSON response and turns it into a Python dict stored in a variable

    #______________ Extracts The Clean Condition Dealer Retail Value From JSON Response __________________________________
    
    # Market data payload stored as a variable to make it easier to call 
    market_data_payload = info["data"]["market_value"]["market_value_data"]

    # Loop through the different market conditions, and looks for the "Clean" condition and returns the "Dealer Retail" value
    for entry in market_data_payload:
        for cond in entry["market value"]:
            if cond["Condition"] == "Clean":
                clean_retail = dollars_to_int(cond['Dealer Retail']) # Stores our desired value into a variable
                break        # found it → stop inner loop
        else:
            continue # if 'Clean' condition is not found in this entry it will loop into the next condition dictionary
        break # if 'Clean' condition is found we exit into the outer loop
    else:
        raise KeyError(" Clean condition dealer retail price was not present in JSON response")
    # Exits the loop with the Clean Condition Retail Value and stores it in an object
    return {
            #"VIN": info["data"]["intro"]["vin"],
            "YEAR": info["data"]["basic"]["year"],
            "MAKE": info["data"]["basic"]["make"],
            "MODEL": info["data"]["basic"]["model"],
            "TRIM": info["data"]["basic"]["trim"],
            "CLEAN_MARKET_VALUE" : clean_retail   
        }

# (Part 3) API Call Loop 
Where we make our License Plate -> VIN call and then use our get_market_value() function to do the second part of VIN -> Market Value. The first loop will go through the list of license plates with all 50 states and call the API to get a VIN, if the license plate / state is invalid it will go down the list of all 50 states. If the all plate / state combinations failed, it will skip to the next plate in the dataframe before using our get_market_value() function. This ensures that only valid license plates go into the market value API call and returns only information for vaild license plates. 


In [ ]:
BASE_URL = "https://api.vehicledatabases.com/license-decode"

# Only loops through the closest states
STATE_CODES = ["CA", "AZ", "NV", "OR", "WA"]

total_plates = len(plate_df) # Tells how many plates that will be iterated

bar_width = 50 # Sets the bar width for the progress bar                          

http = requests.Session()

for idx, (_, row) in enumerate(plate_df.iterrows(), start=1):
    # ---- progress bar ----
    filled   = int(bar_width * idx / total_plates)
    bar      = "█" * filled + "-" * (bar_width - filled)
    percent  = 100 * idx / total_plates
    
    print(f"\r|{bar}| {idx}/{total_plates} ({percent:5.1f}%)", end="", flush=True)
    
    plate_number = re.sub(r"\s+", "", row['LICENSE_PLATE_NUMBER'])

    # loop through every state until one returns a VIN
    for STATE in STATE_CODES:
        plate_info = {
            "PLATE": plate_number,
            "STATE": STATE,
            "VIN":"",
            "ERROR":"",
            "IMPORT_DATE": today
        } # Dictionary containing all the colum info we want to pull for each row that will later be added after the VIN and Market-Value API calls
        url     = f"{BASE_URL}/{plate_info['PLATE']}/{plate_info['STATE']}"
        headers = {"x-AuthKey" : AUTH_KEY}
    
# __________________________________Part-One, License Plate -> VIN Call ___________________________________________________
    
        try:
            r = http.get(url, headers=headers, timeout=20) # Sets up the get request
            r.raise_for_status() # Looks at the HTTP status code and will keep going or raise and error
            info = r.json() # Takes the API's JSON response and transforms it into a Python dictionary
            vin = info["data"]["intro"]["vin"] # Saves the VIN from API's JSON response as a variable to be used in Part-Two of the code
            make = info["data"]["basic"]["make"] # Saves the make,model,year from the API's JSON response into it's own variable, incase the second API call fails, the data will still be added to the results table.
            model = info["data"]["basic"]["model"]
            year = info["data"]["basic"]["year"]
            plate_info["VIN"] = vin # Saving the vin in case the second loop fails
            print(f"1️⃣  Success Calling License Plate -> VIN for Plate: {plate_info['PLATE']} State: {plate_info['STATE']}, VIN is: {vin}") # Confirmation statement, printing the vehicle info we stored in our results[] 
            break
            
        # Invalid combination of license plate number and state, will try the next state     
        except (requests.exceptions.RequestException, KeyError) as err:
           # print(f"⚠️  No VIN for plate {plate_number} in {plate_info['STATE']}: {err}")
            time.sleep(0.00) # respect rate-limit
            continue         
    else:
        #plate_info.update({"ERROR": "vin lookup failed for every state"})
    
        # Calls the save_plate function to save the error directly into Snowflake
        plate_info.update({"STATE":""})
        save_plate(plate_info)
        
        # Add an error code for when it goes through all plates but doesn't return a value
        print(f"⚠️  License Plate Number {plate_number} is Invalid, Moving Onto Next License Plate Number")
    
        time.sleep(0.00)
        continue                             
# __________________________________Part-Two, VIN -> Market Value Function Call__________________________________________
        
        # Attempts to call our get_market_value() function using the VIN pulled from the first part of our loop
    try:
        rows = get_market_value(vin) # activates our market value lookup function and stores the results into an variable called rows
        #print(f"2️⃣  Success Calling VIN -> Market Value for Plate: {plate_info['PLATE']} State: {plate_info['STATE']} & VIN:{vin}")
        
        rows.update(plate_info) # adds the key / value pairs of STATE and LICENSE PLATE NUMBER to our function results
        
        save_plate(rows) # adds to the end of our list the data dictionary with vehicle info returned from the get_market_value() function
        
        print(f"✅  Complete Successful Call for Plate: {plate_info['PLATE']} / {plate_info['STATE']} & VIN: {plate_info['VIN']}")    
        
        # Error exceptions in case anything happens   
    except requests.exceptions.RequestException as mv_err:
        print(f"⚠️  Market-Value API Error For {vin}: {mv_err}")
        plate_info.update({"ERROR": f"market API: {mv_err}"})
        save_plate(plate_info)
        
    except KeyError as data_err: # “Clean Condition Dealer Retail” value is missing, etc.
        print(f"⚠️  Market-Value API Error, Clean Condition Dealer Retail Data is Missing For {vin}: {data_err}")
        plate_info.update({"ERROR": "average dealer retail not found"})
        save_plate(plate_info)
        
    except Exception as unknown_err:   # last-resort safety net
        print(f"💥 unexpected error for {vin}: {unknown_err}")
        plate_info.update({"ERROR":f" unknown Error: {unknown_err}"})
        save_plate(plate_info)
        
    #print("Moving onto next license plate")

session.close()
print("Finished itterating through all plates")